In [2]:
!python3 -m pip install --upgrade pip

    100% |################################| 1.7MB 300kB/s eta 0:00:01
  Found existing installation: pip 9.0.1
    Not uninstalling pip at /usr/lib/python3/dist-packages, outside environment /usr


In [3]:
!pip install torchvision timm

     |################################| 549 kB 11.7 MB/s            
     |################################| 67 kB 2.0 MB/s            
     |################################| 78 kB 3.0 MB/s             


In [6]:
# 导入必要的库
import torch
import timm

# 定义加载模型的函数
def load_efficientnet_b0(model_path, num_classes=39, device=None):
    """
    加载保存的 EfficientNet-B0 模型
    
    参数:
        model_path: 模型权重文件路径，字符串类型
        num_classes: 分类类别数量，整数类型，默认为39
        device: 计算设备，默认为自动检测
    
    返回:
        model: 加载好的模型对象
    """
    # 检测设备
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 创建与训练时相同的模型结构
    model = timm.create_model('efficientnet_b0', pretrained=False, num_classes=num_classes)
    
    # 加载模型权重
    model.load_state_dict(torch.load(model_path, map_location=device))
    
    # 将模型移至指定设备并设置为评估模式
    model = model.to(device)
    model.eval()
    
    return model

# 使用示例

model_path = "EfficientNet-B0.pth"
    
    # 加载模型
model = load_efficientnet_b0(model_path)
print("模型已成功加载")

模型已成功加载


In [7]:
# 植物病害类别列表（39类，最后一类是背景）
# Plant disease class list (39 classes, last is 'background')
disease_names = [
    'Apple___Apple_scab',  # 苹果黑星病 Apple scab
    'Apple___Black_rot',  # 苹果黑腐病 Apple Black rot
    'Apple___Cedar_apple_rust',  # 苹果雪松锈病 Apple Cedar apple rust
    'Apple___healthy',  # 苹果健康 Apple healthy
    'Blueberry___healthy',  # 蓝莓健康 Blueberry healthy
    'Cherry_(including_sour)___Powdery_mildew',  # 樱桃白粉病 Cherry Powdery mildew
    'Cherry_(including_sour)___healthy',  # 樱桃健康 Cherry healthy
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',  # 玉米灰斑病 Corn Gray leaf spot
    'Corn_(maize)___Common_rust_',  # 玉米普通锈病 Corn Common rust
    'Corn_(maize)___Northern_Leaf_Blight',  # 玉米北方叶斑病 Corn Northern Leaf Blight
    'Corn_(maize)___healthy',  # 玉米健康 Corn healthy
    'Grape___Black_rot',  # 葡萄黑腐病 Grape Black rot
    'Grape___Esca_(Black_Measles)',  # 葡萄腐烂病 Grape Esca (Black Measles)
    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',  # 葡萄叶枯病 Grape Leaf blight (Isariopsis Leaf Spot)
    'Grape___healthy',  # 葡萄健康 Grape healthy
    'Orange___Haunglongbing_(Citrus_greening)',  # 橙黄龙病 Orange Huanglongbing (Citrus greening)
    'Peach___Bacterial_spot',  # 桃细菌性斑点 Peach Bacterial spot
    'Peach___healthy',  # 桃健康 Peach healthy
    'Pepper,_bell___Bacterial_spot',  # 灯笼椒细菌性斑点 Pepper, bell Bacterial spot
    'Pepper,_bell___healthy',  # 灯笼椒健康 Pepper, bell healthy
    'Potato___Early_blight',  # 马铃薯早疫病 Potato Early blight
    'Potato___Late_blight',  # 马铃薯晚疫病 Potato Late blight
    'Potato___healthy',  # 马铃薯健康 Potato healthy
    'Raspberry___healthy',  # 树莓健康 Raspberry healthy
    'Soybean___healthy',  # 大豆健康 Soybean healthy
    'Squash___Powdery_mildew',  # 南瓜白粉病 Squash Powdery mildew
    'Strawberry___Leaf_scorch',  # 草莓叶灼病 Strawberry Leaf scorch
    'Strawberry___healthy',  # 草莓健康 Strawberry healthy
    'Tomato___Bacterial_spot',  # 番茄细菌性斑点 Tomato Bacterial spot
    'Tomato___Early_blight',  # 番茄早疫病 Tomato Early blight
    'Tomato___Late_blight',  # 番茄晚疫病 Tomato Late blight
    'Tomato___Leaf_Mold',  # 番茄叶霉病 Tomato Leaf Mold
    'Tomato___Septoria_leaf_spot',  # 番茄叶斑病 Tomato Septoria leaf spot
    'Tomato___Spider_mites Two-spotted_spider_mite',  # 番茄二斑叶螨 Tomato Spider mites (Two-spotted spider mite)
    'Tomato___Target_Spot',  # 番茄靶斑病 Tomato Target Spot
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus',  # 番茄黄化卷叶病毒病 Tomato Yellow Leaf Curl Virus
    'Tomato___Tomato_mosaic_virus',  # 番茄花叶病毒病 Tomato mosaic virus
    'Tomato___healthy',  # 番茄健康 Tomato healthy
    'background'  # 背景背景（非植物）Background (non-plant)
]


In [15]:
!ls -l /dev/video*

crw-rw---- 1 root video 81, 0 May  1 00:54 /dev/video0


In [12]:
import cv2
import numpy as np
from IPython.display import clear_output, Image, display
import matplotlib.pyplot as plt
%matplotlib inline

def display_camera_feed():
    # 创建VideoCapture对象
    cap = cv2.VideoCapture(0)
    
    # 检查摄像头是否成功打开
    if not cap.isOpened():
        print("无法打开摄像头")
        return
    
    try:
        while True:
            # 读取一帧图像
            ret, frame = cap.read()
            
            if ret:
                # 转换颜色空间从 BGR 到 RGB
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # 清除之前的输出
                clear_output(wait=True)
                
                # 使用 matplotlib 显示图像
                plt.figure(figsize=(10,8))
                plt.imshow(frame_rgb)
                plt.axis('off')
                plt.show()
                
                # 短暂延时
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                print("无法读取视频帧")
                break
                
    finally:
        # 释放资源
        cap.release()
        plt.close()

if __name__ == "__main__":
    display_camera_feed()



无法打开摄像头


In [16]:
import cv2

# 直接使用设备路径
cap = cv2.VideoCapture("/dev/video0")

if not cap.isOpened():
    print("无法打开摄像头")
else:
    print("摄像头已成功打开")
    ret, frame = cap.read()
    if ret:
        print(f"成功读取图像帧，尺寸: {frame.shape}")
    else:
        print("无法读取图像帧")
    
    cap.release()

无法打开摄像头


In [9]:
# 导入必要的库
import cv2
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
import time

# 定义图像预处理转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整大小为 224x224
    transforms.ToTensor(),          # 转换为张量
])

# 定义预测函数
def predict_frame(model, frame, transform, device, top_k=3):
    """
    预测单帧图像的植物病害类别
    
    参数:
        model: 加载好的模型
        frame: OpenCV的BGR图像帧
        transform: 预处理转换
        device: 计算设备
        top_k: 返回概率最高的前k个结果
    
    返回:
        results: 包含(类别索引, 类别名称, 置信度)的列表
    """
    # 将BGR图像转换为RGB，然后转为PIL图像
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_frame)
    
    # 预处理图像
    image_tensor = transform(pil_image).unsqueeze(0).to(device)  # 添加批次维度
    
    # 预测
    with torch.no_grad():  # 不计算梯度以提高速度
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)[0]
    
    # 获取前k个预测结果
    top_probs, top_indices = torch.topk(probabilities, top_k)
    
    # 构建结果列表
    results = []
    for i in range(top_k):
        idx = top_indices[i].item()
        results.append((idx, disease_names[idx], top_probs[i].item()))
    
    return results

# 摄像头实时预测函数
def camera_predict():
    # 获取设备
    device = next(model.parameters()).device
    
    # 初始化摄像头
    cap = cv2.VideoCapture(0)  # 0表示默认摄像头
    
    if not cap.isOpened():
        print("无法打开摄像头，请检查摄像头连接")
        return
    
    # 设置帧率控制和计算FPS的变量
    prev_time = time.time()
    fps = 0
    
    print("按 'q' 键退出")
    
    while True:
        # 捕获一帧图像
        ret, frame = cap.read()
        if not ret:
            print("无法获取图像帧")
            break
        
        # 计算FPS
        curr_time = time.time()
        elapsed = curr_time - prev_time
        prev_time = curr_time
        fps = 1 / elapsed if elapsed > 0 else 0
        
        # 获取预测结果
        predictions = predict_frame(model, frame, transform, device)
        
        # 创建覆盖层显示预测信息
        overlay = frame.copy()
        cv2.rectangle(overlay, (10, 10), (400, 140), (0, 0, 0), -1)  # 黑色背景框
        
        # 设置文本参数
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.6
        font_thickness = 1
        
        # 显示FPS
        fps_text = f"FPS: {fps:.1f}"
        cv2.putText(overlay, fps_text, (20, 30), font, font_scale, (0, 255, 0), font_thickness)
        
        # 显示预测结果
        y_offset = 60
        for idx, disease_name, confidence in predictions:
            # 提取植物名称和状态
            parts = disease_name.split('___')
            plant = parts[0].replace('_', ' ')
            condition = parts[1].replace('_', ' ') if len(parts) > 1 else ''
            
            # 根据植物状态设置颜色
            if "healthy" in disease_name:
                color = (0, 255, 0)  # 绿色表示健康
            elif confidence > 0.7:
                color = (0, 0, 255)  # 红色表示高置信度的疾病
            else:
                color = (255, 165, 0)  # 橙色表示中等置信度
            
            text = f"{plant}: {condition} ({confidence:.2f})"
            cv2.putText(overlay, text, (20, y_offset), font, font_scale, color, font_thickness)
            y_offset += 25
        
        # 将覆盖层与原图像混合
        alpha = 0.7  # 透明度
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
        
        # 显示实时画面
        cv2.imshow('植物病害实时检测', frame)
        
        # 按q键退出
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # 释放资源
    cap.release()
    cv2.destroyAllWindows()

# 运行摄像头预测
camera_predict()

无法打开摄像头，请检查摄像头连接


In [ ]:
# 39 plant disease classes with prevention & treatment suggestions (English only)
treatment_dict = {
    "Apple___Apple_scab": "Remove diseased leaves/fruits, improve ventilation, and spray appropriate fungicides.",
    "Apple___Black_rot": "Prune infected branches and apply fungicides during autumn/winter cleanup.",
    "Apple___Cedar_apple_rust": "Plant resistant varieties, prune regularly, and prevent cross-infection.",
    "Apple___healthy": "Plant is healthy, no treatment needed.",
    "Blueberry___healthy": "Plant is healthy, no treatment needed.",
    "Cherry_(including_sour)___Powdery_mildew": "Increase ventilation, remove infected leaves, and spray fungicides like myclobutanil.",
    "Cherry_(including_sour)___healthy": "Plant is healthy, no treatment needed.",
    "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot": "Use resistant varieties, remove infected residues, and practice crop rotation.",
    "Corn_(maize)___Common_rust_": "Remove diseased plants promptly and apply fungicides like triadimefon.",
    "Corn_(maize)___Northern_Leaf_Blight": "Use resistant varieties, enhance field management, and apply fungicides in time.",
    "Corn_(maize)___healthy": "Plant is healthy, no treatment needed.",
    "Grape___Black_rot": "Prune diseased branches, clean fallen leaves/fruits, and spray Bordeaux mixture.",
    "Grape___Esca_(Black_Measles)": "Prune and burn infected branches, maintain vineyard hygiene.",
    "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)": "Remove infected leaves, prune properly, and apply fungicides.",
    "Grape___healthy": "Plant is healthy, no treatment needed.",
    "Orange___Haunglongbing_(Citrus_greening)": "Remove infected trees, control psyllid vectors, and use healthy seedlings.",
    "Peach___Bacterial_spot": "Avoid wounds, plant resistant varieties, and spray copper-based fungicides.",
    "Peach___healthy": "Plant is healthy, no treatment needed.",
    "Pepper,_bell___Bacterial_spot": "Remove diseased plants, improve management, and apply copper fungicides.",
    "Pepper,_bell___healthy": "Plant is healthy, no treatment needed.",
    "Potato___Early_blight": "Practice crop rotation, plant properly, and apply chlorothalonil fungicide.",
    "Potato___Late_blight": "Use resistant varieties, apply fungicides promptly, and avoid field water accumulation.",
    "Potato___healthy": "Plant is healthy, no treatment needed.",
    "Raspberry___healthy": "Plant is healthy, no treatment needed.",
    "Soybean___healthy": "Plant is healthy, no treatment needed.",
    "Squash___Powdery_mildew": "Improve ventilation, control humidity, and spray sulfur-based fungicides.",
    "Strawberry___Leaf_scorch": "Remove infected leaves, avoid excessive moisture, and use fungicides when needed.",
    "Strawberry___healthy": "Plant is healthy, no treatment needed.",
    "Tomato___Bacterial_spot": "Use healthy seedlings, remove diseased leaves, and spray copper fungicides.",
    "Tomato___Early_blight": "Practice crop rotation, spray fungicides promptly, and improve ventilation.",
    "Tomato___Late_blight": "Plant resistant varieties, avoid high humidity, and apply fungicides promptly.",
    "Tomato___Leaf_Mold": "Control humidity, remove diseased leaves, and use special fungicides.",
    "Tomato___Septoria_leaf_spot": "Proper spacing, reduce water splash, and spray fungicides timely.",
    "Tomato___Spider_mites Two-spotted_spider_mite": "Apply acaricides and keep the field clean.",
    "Tomato___Target_Spot": "Improve ventilation and apply fungicides promptly.",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus": "Remove diseased plants, control aphids, and plant resistant varieties.",
    "Tomato___Tomato_mosaic_virus": "Avoid mechanical injuries and use healthy seedlings.",
    "Tomato___healthy": "Plant is healthy, no treatment needed.",
    "background": "No plant or disease detected."
}


In [8]:
# 打开摄像头，0表示第一个摄像头设备
# Open camera, 0 means the first camera device
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # 读取一帧图像 Read one frame
    if not ret:
        break

    # 病害识别 Predict disease class
    class_id, conf_score = detect_disease(frame)

    # 画方框 Draw rectangle in the center
    h, w, _ = frame.shape
    pt1 = (w // 4, h // 4)
    pt2 = (w * 3 // 4, h * 3 // 4)
    cv2.rectangle(frame, pt1, pt2, (0, 255, 0), 2)  # 在中心画一个绿色方框 Draw a green rectangle at the center

    # 构造标签文本 Class label text
    label_text = f"{disease_names[class_id]}: {conf_score:.2f}"

    # 查找防治建议，Get treatment suggestion
    suggestion = treatment_dict.get(disease_names[class_id], "No suggestion available.")

    # 在画面左上角写出分类和置信度 Put class/confidence on the top left
    cv2.putText(frame, label_text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    # 在画面下方分行显示防治建议（建议较长自动换行） Put treatment suggestion at the bottom (with word wrapping)
    max_chars_per_line = 50  # 每行最多字数 Max chars per line
    suggestion_lines = [suggestion[i:i+max_chars_per_line] for i in range(0, len(suggestion), max_chars_per_line)]
    for i, line in enumerate(suggestion_lines):
        y_pos = h - 60 + i * 30  # 行距 Line spacing
        cv2.putText(frame, line, (20, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,128,255), 2)

    # 实时显示画面 Real-time show
    cv2.imshow("Plant Disease Detection 植物病害识别", frame)

    # 按q键退出，Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()  # 释放摄像头 Release camera
cv2.destroyAllWindows()  # 关闭所有窗口 Close all windows


KeyboardInterrupt: 